In [1]:
import pandas as pd

card_infos = pd.read_csv('./shinhan_cardInfos.csv')

card_urls = card_infos['card_url'].tolist()
card_names = card_infos['card_name'].tolist()
card_imgs = card_infos['card_img'].tolist()

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

card_summary = []
card_benefit = []

for url in card_urls:
    url = f'https://www.shinhancard.com/pconts/html/card/apply/credit/{url}'

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    driver.implicitly_wait(20)
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 혜택 요약 =============================================
    all_view = soup.find('ul', {'class':'all_view'})
    strong_list = all_view.find_all('strong') # 혜택 제목 리스트
    summary = ''
    for i in range(len(strong_list)):
        summary += f'[{strong_list[i].text}]'
        ul_tag = strong_list[i].find_next('ul', class_='marker_dot')
        li_tags = ul_tag.find_all('li')

        for j in range(len(li_tags)):
            summary += f'{i + 1}. {li_tags[j].text}'
        
    tap_wrap = soup.find('div', {'class':'tab_wrap'})
    hidden_tap = tap_wrap.find_all('li', {'aria-hidden':'true'})

    # 상세 혜택 =============================================    
    benefit_title = [] # 탭 제목들 (혜택 제목)
    benefit = ''
    for i in range(len(hidden_tap)):
        benefit_title.append(hidden_tap[i].find('h2', {'class':'hidden-text'}).text.strip())

    for i in range(len(hidden_tap)):
        benefit += f'###{benefit_title[i]}'
        tit_dep2 = hidden_tap[i].find_all(class_="tit_dep2")

        for title in tit_dep2:
            benefit += f'[{title.text.strip()}]'
            next_sibling = title.find_next_sibling()
            while next_sibling:
                if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                    table = next_sibling.select_one('table')
                    # 표가 있을 때만 출력
                    if table:
                        rows = table.find_all('tr')
                        benefit += table.find('strong').text.strip()
                        for j, row in enumerate(rows):

                            cells = row.find_all(['th', 'td'])
                            row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                            row_string = ' '.join(row_data)

                            # 각 행을 설명하는 문장 출력
                            sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                            benefit += sentence
                            
                elif next_sibling.name == 'p':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                        benefit += next_sibling.text.strip()
                elif next_sibling.name == 'ul':
                    class_list = next_sibling.get('class', [])
                    if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                        li_list = next_sibling.find_all('li', recursive=False)
                        for li in li_list:
                            benefit += li.text.strip()

                # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                if 'tit_dep2' in next_sibling.get('class', []) or ('h4' in next_sibling.name and 'tit_dep3' in next_sibling.get('class', [])):
                    break
                
                next_sibling = next_sibling.find_next_sibling()
        
        tit_dep3 = hidden_tap[i].find_all('h4', class_="tit_dep3")
        if tit_dep3:
            for title in tit_dep3:
                benefit += f'[{title.text.strip()}]'
                next_sibling = title.find_next_sibling()
                while next_sibling:
                    if next_sibling.name == 'div' and 'table_wrap' in next_sibling.get('class', []):
                        table = next_sibling.select_one('table')
                        # 표가 있을 때만 출력
                        if table:
                            rows = table.find_all('tr')
                            benefit += table.find('strong').text.strip()
                            for j, row in enumerate(rows):

                                cells = row.find_all(['th', 'td'])
                                row_data = ['(' + cell.text.strip() + ')' for cell in cells]
                                row_string = ' '.join(row_data)

                                # 각 행을 설명하는 문장 출력
                                sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                                benefit += sentence
                    elif next_sibling.name == 'p':
                        class_list = next_sibling.get('class', [])
                        if ('marker_dot' in class_list) or ('marker_refer' in class_list):
                            benefit += next_sibling.text.strip()
                            
                    elif next_sibling.name == 'ul':
                        class_list = next_sibling.get('class', [])
                        if ('marker_dot' in class_list) or ('marker_refer' in class_list) or ('marker_hyphen' in class_list):
                            li_list = next_sibling.find_all('li', recursive=False)
                            for li in li_list:
                                benefit += li.text.strip()

                    # 다음 형제 요소 찾을 때 특정 조건을 만족하면 루프 종료
                    if 'tit_dep3' in next_sibling.get('class', []):
                        break
                    
                    next_sibling = next_sibling.find_next_sibling()
                    
    card_summary.append(summary)
    card_benefit.append(benefit)
    
driver.quit()

data = {"card_names": card_names, "card_imgs": card_imgs, "card_summarys": card_summary, "card_benefits" : card_benefit}
df = pd.DataFrame(data)

df.to_csv("shinhan_benefits.csv", encoding = "utf-8-sig")

In [3]:
shinhan_benefits = pd.read_csv('./shinhan_benefits.csv')
shinhan_benefits

,Unnamed: 0,card_names,card_imgs,card_summarys,card_benefits
0,0,신한카드 봄,https://www.shinhancard.com/pconts/images/cont...,[기본 서비스]1. 국내외 이용금액 최대 0.5% 캐시백1. 분기 5천원 추가 캐시...,###기본 서비스[월 캐시백]국내외 이용금액 캐시백전월 이용금액에 따른 월 캐시백율...
1,1,신한카드 봄(도구리),https://www.shinhancard.com/pconts/images/cont...,[기본 서비스]1. 국내외 이용금액 최대 0.5% 캐시백1. 분기 5천원 추가 캐시...,###기본 서비스[월 캐시백]국내외 이용금액 캐시백전월 이용금액에 따른 월 캐시백율...
2,2,신한카드 Globus,https://www.shinhancard.com/pconts/images/cont...,[해외 수수료 할인]1. 국제브랜드사 수수료 1% 할인1. 해외서비스 수수료 0.1...,###해외 수수료 할인[해외 수수료 할인 서비스]국제브랜드사 수수료 1% 할인해외서...
3,3,신한카드 플리,https://www.shinhancard.com/pconts/images/cont...,[Daily 결제일 할인]1. 국내 이용금액 최대 0.9% 결제일 할인\n ...,###Daily 결제일 할인[Daily 결제일 할인]국내 이용금액 최대 0.9% 결...
4,4,신한카드 플리(산리오캐릭터즈),https://www.shinhancard.com/pconts/images/cont...,[Daily 결제일 할인]1. 국내 이용금액 최대 0.9% 결제일 할인\n ...,###Daily 결제일 할인[Daily 결제일 할인]국내 이용금액 최대 0.9% 결...
...,...,...,...,...,...
89,89,교직원복지 신한카드 LOVE,https://www.shinhancard.com/pconts/images/cont...,[LG계열 할인]1. LG전자 5% 할인1. 뷰티플렉스 10% 할인[쇼핑 할인]2....,"###LG계열 할인[LG계열 제휴사 할인 및 포인트 적립]LG전자(하이프라자, LG..."
90,90,한국수력원자력 임직원복지 신한카드 LOVE,https://www.shinhancard.com/pconts/images/cont...,"[쇼핑 할인]1. 현대, 롯데, 신세계, 갤러리아 5% 할인1. 이마트, 롯데마트,...","###쇼핑 할인[국내 유명 백화점, 할인점, 온라인 쇼핑몰 최대 5% 할인]백화점:..."
91,91,SK One+ 화물복지 신한카드,https://www.shinhancard.com/pconts/images/cont...,[유가보조금 지원]1. 전국 주유소에서 주유 시 화물운전자 유가보조금 자동 지급[주...,###유가보조금 지원[화물운전자 복지카드(유류구매카드)란?]국토교통부가 시행하는 [...
92,92,부산가족사랑 GS칼텍스 신한카드 Shine,https://www.shinhancard.com/pconts/images/cont...,"[주유 할인]1. CGV, 메가박스 온라인 영화 예매 할인 1. 뷰티 할인1. 프로...",###주유 할인[GS칼텍스 주유 시 리터당 60~100원 청구할인(LPG제외)]1회...
